# Importing

In [11]:
# # keep runtime
# function ClickConnect(){
# console.log("Working"); 
# document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
# }
# var clicker = setInterval(ClickConnect,60000);
# # stop keeping runtime 
# clearInterval(clicker);

from google.colab import drive
drive.mount('/content/drive')

!pip install catboost
!pip install tensorflow-addons
!pip install keras-tuner
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/projects/DA_Platform/")
from DA_v5 import *

!pip install wandb
import wandb
from wandb.keras import WandbCallback
import shutil

# global setting
warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found GPU at: /device:GPU:0


In [12]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/"
AUTOTUNE = tf.data.experimental.AUTOTUNE
img_size = 224
channels = 3
target_var = "growth_day"
plant_type = "LT"

# Image loading and preprocessing functions

In [13]:
from glob import glob
from IPython.display import Image

def extract_day(file_name):
    # 01, 02 등으로 된 문자열을 int로 변환하여 성장일 추출
    return int(file_name.split('.')[-2][-2:])

def make_train_dataframe(root_path=None, plant_type=None):
    # ex. /train_dataset/BC/_01, /train_dataset/BC/_02 등
    image_dir = glob(root_path + plant_type + "/*")
    image_dir = series(image_dir).sort_values()

    before_path = []
    after_path = []
    growth_day = []
    for i in image_dir: 
        images = glob(i + '/*.png')
        images = series(images).sort_values()
        cnt = 0
        for j in images.iloc[cnt:]:
            if len(images) - 1 == cnt: break
            for k in images.iloc[cnt+1:]:
                before_path.append(j)
                after_path.append(k)     
                growth_day.append(extract_day(k) - extract_day(j))
            cnt += 1

    result_df = dataframe({
        "before_path": before_path,
        "after_path": after_path,
        "growth_day": growth_day
    })
    return result_df

def make_test_dataframe(root_path=None, plant_type=None):
    test_metadata = read_csv(root_path + "test_data.csv")

    tmp_plant_type = []
    test_metadata.iloc[:,1].apply(lambda x: tmp_plant_type.append(x.split("_")[1]))
    test_metadata = test_metadata.drop("idx", axis=1)[series(tmp_plant_type).apply(lambda x: x == plant_type)]
    test_plant_idx = test_metadata.reset_index()["index"]; test_metadata.reset_index(drop=True, inplace=True)

    before_file_path = []
    after_file_path = []
    for i in range(test_metadata.shape[0]):
        path_data = test_metadata["before_file_path"].iloc[i].split("_")
        file_path = root_path + path_data[1] + "/" + path_data[2] + "/" + test_metadata["before_file_path"].iloc[i] + ".png"
        before_file_path.append(file_path)

        path_data = test_metadata["after_file_path"].iloc[i].split("_")
        file_path = root_path + path_data[1] + "/" + path_data[2] + "/" + test_metadata["after_file_path"].iloc[i] + ".png"
        after_file_path.append(file_path)

    test_metadata["before_file_path"] = before_file_path
    test_metadata["after_file_path"] = after_file_path
    test_metadata.columns = ["before_path", "after_path"]
    return test_metadata, test_plant_idx

# rawdata load and data check

In [14]:
full_x = make_train_dataframe(folder_path + "dataset/train_dataset/", plant_type)
full_y = full_x["growth_day"].astype("float32"); full_x.drop("growth_day", axis=1, inplace=True)
test_x, test_plant_idx = make_test_dataframe(folder_path + "dataset/test_dataset/", plant_type)

In [15]:
print(full_x.shape)
print(full_y.shape)
print(test_x.shape)

(7801, 2)
(7801,)
(2161, 2)


In [16]:
full_x.head(10)

,before_path,after_path
0,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT01.png
1,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT02.png
2,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT03.png
3,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT04.png
4,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT05.png
5,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT06.png
6,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT07.png
7,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT08.png
8,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT09.png
9,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT00.png,/content/drive/MyDrive/Colab Notebooks/projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/LT/LT_01/DAT10.png


In [ ]:
full_x.tail(10)

,before_path,after_path
6802,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT26.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT27.png
6803,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT26.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT28.png
6804,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT26.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT29.png
6805,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT26.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT30.png
6806,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT27.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT28.png
6807,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT27.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT29.png
6808,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT27.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT30.png
6809,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT28.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT29.png
6810,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT28.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT30.png
6811,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT29.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/train_dataset/BC/BC_09/DAT30.png


In [ ]:
# Image(full_x.iloc[0,0])

In [ ]:
# Image(full_x.iloc[0,1])

In [ ]:
test_x.head(10)

,before_path,after_path
0,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00445.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00840.png
1,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1112/idx_BC_1112_00229.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1112/idx_BC_1112_00105.png
2,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1112/idx_BC_1112_00105.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1112/idx_BC_1112_00096.png
3,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00511.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00382.png
4,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00120.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00415.png
5,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00120.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00181.png
6,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00794.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00711.png
7,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00474.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00382.png
8,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00812.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00614.png
9,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1112/idx_BC_1112_00518.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1112/idx_BC_1112_00430.png


In [ ]:
test_x.tail(10)

,before_path,after_path
1789,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00465.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00561.png
1790,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00716.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00443.png
1791,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00713.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00855.png
1792,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00794.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00115.png
1793,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00097.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00036.png
1794,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00148.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00688.png
1795,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00110.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00525.png
1796,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00511.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00132.png
1797,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00353.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1088/idx_BC_1088_00196.png
1798,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00527.png,/content/drive/MyDrive/Colab Notebooks/Projects/Dacon/plant_growth_timeline_prediction/dataset/test_dataset/BC/1100/idx_BC_1100_00640.png


In [ ]:
# Image(test_x.iloc[0,0])

In [ ]:
# Image(test_x.iloc[0,1])

# Image transform to (224, 224)

In [ ]:
# full_x.head(50)

In [17]:
from PIL import Image
import pathlib

image_sources = full_x["before_path"].append(full_x["after_path"]).drop_duplicates().reset_index(drop=True).sort_values()
for i in tqdm(image_sources):
    image_obj = Image.open(i)
    image_obj = image_obj.resize((img_size, img_size), Image.BICUBIC)
    image_resize_path = ["dataset_224" if j == "dataset" else j for j in i.split("/")]
    # make directory
    pathlib.Path("/".join(image_resize_path[:-1])).mkdir(parents=True, exist_ok=True)
    # save resize file
    image_obj.save("/".join(image_resize_path), "PNG")

image_sources = test_x["before_path"].append(test_x["after_path"]).drop_duplicates().reset_index(drop=True).sort_values()
for i in tqdm(image_sources):
    image_obj = Image.open(i)
    image_obj = image_obj.resize((img_size, img_size), Image.BICUBIC)
    image_resize_path = ["dataset_224" if j == "dataset" else j for j in i.split("/")]
    # make directory
    pathlib.Path("/".join(image_resize_path[:-1])).mkdir(parents=True, exist_ok=True)
    # save resize file
    image_obj.save("/".join(image_resize_path), "PNG")

100%|██████████| 168/168 [03:59<00:00,  1.43s/it]


In [19]:
full_x = make_train_dataframe(folder_path + "dataset_224/train_dataset/", plant_type)
full_y = full_x["growth_day"].astype("float32"); full_x.drop("growth_day", axis=1, inplace=True)
test_x, test_plant_idx = make_test_dataframe(folder_path + "dataset_224/test_dataset/", plant_type)

In [20]:
print(full_x.shape)
print(full_y.shape)
print(test_x.shape)

(7801, 2)
(7801,)
(2161, 2)
